In [1]:
import transformers
import pandas as pd
from datasets import Dataset
import datasets

In [2]:
! pip install datasets transformers rouge-score nltk


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=e422c8b8af44966e1f8677915a9b72fe036049463077d2474a3492238b9306cf
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 93 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 0s (7157 kB/s)

78Selecting previously unselected package git-lfs.
(Reading database ... 108827 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking git-lfs (2.9.2-1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up git-lfs (2.9.2-1) ...
7Progress: [ 60%] [##################################..........

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
train_set = pd.read_excel("../input/summarization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_train_en.xlsx")
test_set = pd.read_excel("../input/summarization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_test_en.xlsx")
train_ds = Dataset.from_pandas(train_set)
test_ds = Dataset.from_pandas(test_set)

In [6]:
train_dataset, validation_dataset= train_ds.train_test_split(test_size=0.1).values()
data_all_splits = datasets.DatasetDict({"train":train_dataset,"test":test_ds, "val":validation_dataset})


In [7]:
from datasets import load_metric
metric = load_metric("rouge")

In [8]:
from transformers import AutoTokenizer
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [9]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [10]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_datasets = data_all_splits.map(preprocess_function, batched=True)

  0%|          | 0/275 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/31 [00:00<?, ?ba/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 274291
    })
    test: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11529
    })
    val: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30477
    })
})

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [14]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [16]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

In [ ]:
trainer.push_to_hub()


In [17]:
from transformers import pipeline

hub_model_id = "lilouuch/t5-small-finetuned-xsum"
summarizer = pipeline("summarization", model=hub_model_id)

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

2022-11-01 21:22:37.800731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 21:22:37.801749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 21:22:37.802882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 21:22:37.803653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 21:22:37.804378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 274291
    })
    test: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11529
    })
    val: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30477
    })
})

In [ ]:
summaries=summarizer(tokenized_datasets['test']['Document'])

Your max_length is set to 200, but you input_length is only 171. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)
Your max_length is set to 200, but you input_length is only 188. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=94)
Your max_length is set to 200, but you input_length is only 161. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=80)
Your max_length is set to 200, but you input_length is only 129. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)
Your max_length is set to 200, but you input_length is only 171. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)
Your max_length is set to 200, but you input_length is only 149. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)
Your max_length is set to 200, but you input_length is only 168. You might c